# Mago gestor de desastres
#### Grupo:
César Augusto Lima - 9771525<br>
Igor Antunes Boson Paes - 11200571<br>
Tyago Yuji Teoi - 10299660<br>

O projeto se trata de um minigame no qual um mago precisa resolver os desastres apresentados na tela.<br>
Quadrados mágicos invocados pelo mago se encontram posicionados na rua. 
<br>
O mago, para desfazer os desastres, deve ficar acima do quadrado mágico, focalizar na gema do cajado a energia emanada pelo quadrado (deixar o cajado na mesma cor que o quadrado mágico) e realizar a magia de curta, média ou longa distância (apertar a tecla correta)
#### Controles no teclado:
##### Movimentação:
W para cima<br>
A para a esquerda<br>
D para a direita<br>
S para baixo
##### Cajado:
Seta UP    do teclado para cor BRANCA<br>
Seta LEFT  do teclado para cor VERMELHA<br>
Seta DOWN  do teclado para cor VERDE<br>
Seta RIGHT do teclado para cor AZUL
##### Magias:
Q para curta distância (Roda gigante)<br>
E para média distância (Prédio partido)<br>
R para longa distância (Sol em colapso)<br>

#### Instalação de bibliotecas

In [1]:
!pip install glfw
!pip install PyOpenGL

### Inicialização de bibliotecas


In [2]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
from math import *

### Funções de auxilio geral

In [3]:
# Função que testa se dois arrays possuem valores inteiros iguais
def testaParteInteira(A, B):
    if len(A) == len(B):
        for i in range(len(A)):
            if int(A[i]) != int(B[i]):
                return False
        return True
    else:
        return False
    
# Função básica que multiplica duas matrizes
def multiplica_matriz(a,b):
    m_a = a.reshape(4,4)
    m_b = b.reshape(4,4)
    m_c = np.dot(m_a,m_b)
    c = m_c.reshape(1,16)
    return c

# Matriz que nao altera a matriz de transformação,
# caso seja multiplicada
mat_translationZero = np.array([    1.0,  0.0,  0,    0, 
                                    0.0,  1.0,  0,    0, 
                                    0.0,  0.0, 1.0,   0,
                                      0,    0,   0,   1], np.float32)

### Variaveis globais

### Inicialização do sistema de janela GLFW

In [4]:
# Guarda o RGB do cajado do mago
cajadoRGB = [0, 0, 0]

# Array que guarda a posição e o RGB dos quadrados mágicos
                             #Posicao       R G B
quadradoMagico = np.array([[[-0.6  ,-0.6 ],[1,1,1]],
                           [[ 0.16 ,-0.75],[1,1,1]],
                           [[-0.12 ,-0.95],[1,1,1]]])

# Variável que testa se todos os quadrados mágicos se dissiparam (todos os desastres foram resolvidos)
testeAcabouQuadrado = (testaParteInteira(quadradoMagico[0][1], [0,0,0]) and
                       testaParteInteira(quadradoMagico[1][1], [0,0,0]) and
                       testaParteInteira(quadradoMagico[2][1], [0,0,0]))

# Guarda o RGB do Sol e dos raios
raioSol = [[1.0,0.0,0.0], [1.0,0.3,0.0]]

# Variáveis de posição, rotação, escala e o valor dos acréscimos de cada objeto:
txMago = -0.6;        txMagoInc = 0.25;
tyMago = -0.3;        tyMagoInc = 0.1;
sMago = 1;            sMagoInc = 0.1;
rMago = 0;            rMagoInc = 1;

txPredio0 = 0.0;      txPredio0Inc = 0.05;
tyPredio0 = -0.5;     tyPredio0Inc = 0.10;
sPredio0 = 1;         sPredio0Inc = 0.2;
rPredio0 = pi;        rPredio0Inc = pi/4;

txPredio1 = 0.25;     txPredio1Inc = 0.1;
tyPredio1 = -0.5;     tyPredio1Inc = 0.1;
sPredio1 = 1;         sPredio1Inc = 0.2;
rPredio1 = 0;         rPredio1Inc = 1;

txSol0 = 0;           txSol0Inc = 0.1;
tySol0 = 1.1;         tySol0Inc = 0.1;
sSol0 = 1;            sSol0Inc = 0.2;
rSol0 = 0;            rSol0Inc = 1;

txSol1 = 0;           txSol0Inc = 0.1
tySol1 = 1.1;         tySol0Inc = 0.1
sSol1 = 1;            sSol0Inc = 0.02
rSol1 = 0;            rSol0Inc = 1

txNuvem = -0.25;      txNuvemInc = 0.1
tyNuvem = 0.25;       tyNuvemInc = 0.1
sNuvem = 1.5;         sNuvemInc = 0.2
rNuvem = 0;           rNuvemInc = 1

txRoda0 = 0;          txRoda0Inc = 0.1;
tyRoda0 = -0.35;      tyRoda0Inc = 0.1;
sRoda0 = 5;           sRoda0Inc = 0.5;
rRoda0 = 0;           rRoda0Inc = pi/5;

sRoda1 = 3;           sRoda1Inc = 0.3;
sRoda2 = 1;           sRoda2Inc = 0.1;
sRoda3 = 1;           sRoda3Inc = 0.1;

C:\Users\tyago\AppData\Local\Temp/ipykernel_2936/2425299068.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  quadradoMagico = np.array([[[-0.6  ,-0.6 ],[1,1,1]],


In [5]:
glfw.init()

1

### Definição de parâmetros da janela
* Linha 1: janela será criada, mas não será exibida imediatamente
* Linha 2: cria a janela com tamanho (width e height) e título (outras opções disponíveis, como fullscreen ao abrir e controles de monitor
* Linha 3: torna a janela a principal no momento (contexto)

In [6]:
glfw.window_hint (glfw.VISIBLE, glfw.FALSE)
window = glfw.create_window(720, 600, "Projeto 1", None, None)
glfw.make_context_current(window)

### Função para capturar eventos de teclado
A função (key_event) é genérica e apenas imprime no terminal quais teclas estão sendo pressionadas

In [7]:
def key_event(window, key, scancode, action, mods):
    global txMago, tyMago, sMago, rMago, txMagoInc, tyMagoInc, rMagoInc, sMagoInc, cajadoRGB
    global txPredio0, tyPredio0, sPredio0, rPredio0, txPredio0Inc, tyPredio0Inc, rPredio0Inc, sPredio0Inc
    global txPredio1, tyPredio1, sPredio1, rPredio1, txPredio1Inc, tyPredio1Inc, rPredio1Inc, sPredio1Inc 
    global txSol0, tySol0, sSol0, rSol0, txSol0Inc, tySol0Inc, rSol0Inc, sSol0Inc, raioSol
    global txSol1, tySol1, sSol1, rSol1, txSol1Inc, tySol1Inc, rSol1Inc, sSol1Inc 
    global txNuvem, tyNuvem, sNuvem, rNuvem, txNuvemInc, tyNuvemInc, rNuvemInc, sNuvemInc 
    global txRoda0, tyRoda0, sRoda0, rRoda0, txRoda0Inc, tyRoda0Inc, rRoda0Inc, sRoda0Inc 
    global sRoda1, sRoda1Inc 
    global sRoda2, sRoda2Inc 
    global sRoda3, sRoda3Inc 
    global quadradoMagico    
    global testeAcabouQuadrado
    
    testeAcabouQuadrado = (testaParteInteira(quadradoMagico[0][1], [0,0,0]) and
                                  testaParteInteira(quadradoMagico[1][1], [0,0,0]) and
                                  testaParteInteira(quadradoMagico[2][1], [0,0,0]))
    
    #print('[key event] key=', key)
    #print('[key event] scancode=', scancode)
    #print('[key event] action=', action)
    #print('[key event] mods=', mods)
    #print('-------')
    
    # Variáveis de testes para evitar problemas com float
    testeTxMago = int(txMago * 100)
    testeTyMago = int(tyMago * 100)     
        
    #tecla A
    if key == 65 and action == 1:
        if txMago > -0.6:
            txMago -= txMagoInc
        
    #tecla W
    if key == 87 and action == 1:
        if tyMago < -0.35:
            tyMago += tyMagoInc
            sMago -= sMagoInc
        
    #tecla S
    if key == 83 and action == 1:
        if tyMago > -0.5:
            tyMago -= tyMagoInc
            sMago += sMagoInc
        
    #tecla D
    if key == 68 and action == 1:
        if txMago < 0.6:
            txMago += txMagoInc
        
    #tecla Q
    if key == 81 and action == 1:
        if testeTxMago == -60 and testeTyMago == -30:
            if testaParteInteira(quadradoMagico[0][1], [1,0,0]) and testaParteInteira(cajadoRGB, [1,0,0]):
                if sRoda0 > 1:
                    txRoda0 += txRoda0Inc
                    tyRoda0 -= (0.01)*sRoda0/(2*sRoda0Inc )
                    rRoda0 += rRoda0Inc
                    sRoda0 -= sRoda0Inc
                    sRoda1 -= sRoda1Inc
                    sRoda2 -= sRoda2Inc
                    sRoda3 -= sRoda2Inc
                else:
                    quadradoMagico[0][1] = [0,0,0]
                    cajadoRGB = [0,0,0]
        
       
    #tecla E
    if key == 69 and action == 1:
        if testeTxMago == 15 and testeTyMago == -40 :
            if testaParteInteira(quadradoMagico[1][1], [1,0,0]) and testaParteInteira(cajadoRGB, [1,0,0]):
                if tyPredio0 < 0.4:
                    tyPredio0 += tyPredio0Inc
                else:
                    quadradoMagico[1][1] = [0,1,0]
            elif testaParteInteira(quadradoMagico[1][1], [0,1,0]) and testaParteInteira(cajadoRGB, [0,1,0]):
                if txPredio0 < 0.25:
                    txPredio0 += txPredio0Inc
                else:
                    quadradoMagico[1][1] = [0,0,1]
            elif testaParteInteira(quadradoMagico[1][1], [0,0,1]) and testaParteInteira(cajadoRGB, [0,0,1]):
                if rPredio0 > 0:
                    rPredio0 -= rPredio0Inc
                    if rPredio0 <= 0:
                        quadradoMagico[1][1] = [0,0,0]
                        cajadoRGB = [0,0,0]
        
    #tecla R
    if key == 82 and action == 1:
        if testeTxMago == -9 and testeTyMago == -50:
            if testaParteInteira(quadradoMagico[2][1], [1,0,0]) and testaParteInteira(cajadoRGB, [1,0,0]):
                if sSol0 > 0.5 :
                    sSol0 -= sSol0Inc
                    raioSol[0][1] += 1/26
                    raioSol[1][1] += 0.7/26
                    raioSol[1][2] += 1/26
                else:
                    quadradoMagico[2][1] = [0,0,0]
                    cajadoRGB = [0,0,0]
        
    #tecla UP
    if key == 265 and action == 1:
        cajadoRGB = [1, 1, 1]
    #tecla DOWN
    if key == 264 and action == 1:
        cajadoRGB = [0, 1, 0]
    #tecla RIGHT
    if key == 262 and action == 1:
        cajadoRGB = [0, 0, 1]
    #tecla LEFT
    if key == 263 and action == 1:
        cajadoRGB = [1, 0, 0]
        
    
    
glfw.set_key_callback (window, key_event)

### Função para capturar eventos de mouse
A função (mouse_event) apenas exibe qual o botão do mouse foi pressionado.
Detalhes de como coletar eventos de coordenadas do ponteiro/cursor do mouse: https://www.glfw.org/docs/3.3.2/input_guide.html

In [8]:
# Quantidade de acréscimo atual e acréscimo

def mouse_event(window,button,action,mods):

    #print('[mouse event] button=',button)
    #print('[mouse event] action=',action)
    #print('[mouse event] mods=',mods)
    #print('-------')

    if (button == 0 and action == 1):
        print()
    if (button == 1 and action == 1):
        print()

glfw.set_mouse_button_callback(window,mouse_event)

### Definição do vértice

In [9]:
vertex_code = """
        attribute vec2 position;
        uniform mat4 mat_transformation;
        
        void main(){        
            gl_Position = mat_transformation*vec4(position,0.0,1.0);            
        }
        """

### Definição de cor de um fragmento
* void main() é o ponto de entrada do programa (função principal)
* gl_FragColor é variável especial que determina a cor de um fragmento. Pode ser (linha, triangulo, etc)

In [10]:
fragment_code = """
        uniform vec4 color;
        void main(){            
            gl_FragColor = color;
        }
        """


### Requisita slot para a GPU para programas Vertex e Fragment Shaders

In [11]:
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

### Associação do código-fonte aos slots solicitados

In [12]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilação do Vertex Shader

In [13]:
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

### Compilação do Fragment Shader

In [14]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Associação dos programas compilado ao programa principal

In [15]:
glAttachShader(program, vertex)
glAttachShader(program, fragment)

### Link do programa

In [16]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

### Preparando dados para enviar a GPU

In [17]:
# preparando espaço para 20 vértices usando 2 coordenadas (x,y)
num_v = 36
n_verticesCabecaMago = 64 # vértices da cabeça do mago
n_verticesCorpoMago = 32 # vértices do corpo do mago
n_verticesCetroMago0 = 4 # vértices do cetro do mago
n_verticesCetroMago1 = 8 # vértices do cristal do cetro do mago
n_verticesPredioDestruido0 = 18 # vértices do prédio caído
n_verticesPredioDestruido1 = 15 # vertices do prédio incompleto
n_verticesSol0 = 8 # vértices do sol
n_verticesSol1 = 16 # vértices dos raios de sol
n_verticesNuvem = 20 # vértices da núvem

n_verticesTotal = (num_v + n_verticesCabecaMago + n_verticesCorpoMago 
+ n_verticesCetroMago0 + n_verticesCetroMago1 + n_verticesPredioDestruido0
+ n_verticesPredioDestruido1 + n_verticesSol0 + n_verticesSol1 + n_verticesNuvem)

vertices = np.zeros(n_verticesTotal, [("position", np.float32, 2)])

In [18]:
# preenchendo as coordenadas de cada vértice


vertices['position'] = [
                            (0.05 ,0.07),     # Roda Triangulo      vertice 0
                            (0.04 ,0.17),     # Roda Triangulo
                            (0.15 ,0.09),     # Roda Triangulo
                            (-0.05,0.07),     # Roda Triangulo
                            (-0.04,0.17),     # Roda Triangulo
                            (-0.15,0.09),     # Roda Triangulo
                            (0.00,-0.086 ),   # Roda Triangulo
                            (.068,-0.165 ),   # Roda Triangulo
                            (-0.068,-0.165),  # Roda Triangulo
                            (0.078 , -0.015), # Roda Triangulo
                            ( 0.17,0.015 ),   # Roda Triangulo
                            ( 0.13 ,-0.11),   # Roda Triangulo
                            (-0.078,-.015),   # Roda Triangulo
                            (-0.17,0.015 ),   # Roda Triangulo
                            (-0.13 ,-0.11),   # Roda Triangulo
    
                            ( 0.0 , 0.05), # Quadrado Mágico       vertice 15
                            (-0.3 , 0   ), # Quadrado Mágico
                            ( 0 ,-0.09 ),  # Quadrado Mágico
                            ( 0.3 , 0.0 ), # Quadrado Mágico
                            ( 0.0, 0.05 ), # Quadrado Mágico
    
                            (-1.0 ,-1.0 ), # Background cenario 0  vertice 20
                            ( 1.0 ,-1.0 ), # Background cenario 1
                            (-1.0 , 1.0 ), # Background cenario 2
                            ( 1.0 , 1.0 ), # Background cenario 3
                            (-1.0 ,-1.0 ), # Rua cenario 0         vertice 24
                            ( 1.0 ,-1.0 ), # Rua cenario 1
                            (-1.0 ,-0.5 ), # Rua cenario 2
                            ( 1.0 ,-0.5 ), # Rua cenario 3
                            ( 0.7 ,-0.5 ), # Prédio cenario 0      vertice 28
                            ( 1.0 ,-0.5 ), # Prédio cenario 1
                            ( 0.7 , 0.5 ), # Prédio cenario 2
                            ( 1.0 , 0.5 ), # Prédio cenario 3
                            (-1.0 ,-1.0 ), # Prédio janela cenario 0    vertice 32
                            ( 1.0 ,-1.0 ), # Prédio janela cenario 1
                            (-1.0 , 1.0 ), # Prédio janela cenario 2
                            ( 1.0 , 1.0 ),  # Prédio janela cenario 3
    
                            (0 ,0 ), # Mago Cabeça vertice 36
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça
                            (0 ,0 ), # Mago Cabeça 
    
                            (0 ,0 ), # Mago Corpo    vertice 100
                            (0 ,0 ), # Mago Corpo
                            (0 ,0 ), # Mago Corpo
                            (0 ,0 ), # Mago Corpo
                            (0 ,0 ), # Mago Corpo
                            (0 ,0 ), # Mago Corpo
                            (0 ,0 ), # Mago Corpo
                            (0 ,0 ), # Mago Corpo
                            (0 ,0 ), # Mago Corpo
                            (0 ,0 ), # Mago Corpo
                            (0 ,0 ), # Mago Corpo
                            (0 ,0 ), # Mago Corpo
                            (0 ,0 ), # Mago Corpo
                            (0 ,0 ), # Mago Corpo
                            (0 ,0 ), # Mago Corpo
                            (0 ,0 ), # Mago Corpo
                            (0 ,0 ), # Mago Corpo
                            (0 ,0 ), # Mago Corpo
                            (0 ,0 ), # Mago Corpo
                            (0 ,0 ), # Mago Corpo
                            (0 ,0 ), # Mago Corpo
                            (0 ,0 ), # Mago Corpo
                            (0 ,0 ), # Mago Corpo
                            (0 ,0 ), # Mago Corpo
                            (0 ,0 ), # Mago Corpo
                            (0 ,0 ), # Mago Corpo
                            (0 ,0 ), # Mago Corpo
                            (0 ,0 ), # Mago Corpo
                            (0 ,0 ), # Mago Corpo
                            (0 ,0 ), # Mago Corpo
                            (0 ,0 ), # Mago Corpo
                            (0 ,0 ), # Mago Corpo    
    
                            (0.13, -0.3), # Mago Cetro0     vertice 132
                            (0.17, -0.3), # Mago Cetro0
                            (0.13, 0.06), # Mago Cetro0
                            (0.17, 0.06), # Mago Cetro0
    
                            (0, 0), # Mago Cetro1     vertice 136
                            (0, 0), # Mago Cetro1
                            (0, 0), # Mago Cetro1
                            (0, 0), # Mago Cetro1   
                            (0, 0), # Mago Cetro1
                            (0, 0), # Mago Cetro1
                            (0, 0), # Mago Cetro1
                            (0, 0), # Mago Cetro1   
    
                            (0, 0), # Predio destruído caído          vertice 144
                            (0.4, 0), # Predio destruído caído
                            (0,-0.3), # Predio destruído caído
    
                            (0,-0.3), # Predio destruído caído          vertice 147
                            (0.4,-0.3), # Predio destruído caído
                            (0.4, 0), # Predio destruído caído
    
                            (0,-0.3), # Predio destruído caído      vertice 150
                            (0,-0.5), # Predio destruído caído 
                            (0.1,-0.6), # Predio destruído caído
    
                            (0,-0.3), # Predio destruído caído    vertice 153
                            (0.1,-0.6), # Predio destruído caído
                            (0.2,-0.3), # Predio destruído caído   
    
                            (0.2,-0.3), # Predio destruído caído  vertice 156
                            (0.2,-0.6), # Predio destruído caído
                            (0.4,-0.3), # Predio destruído caído
    
                            (0.4,-0.3), # Predio destruído caído vertice 159
                            (0.4,-0.4), # Predio destruído caído
                            (0.2,-0.6), # Predio destruído caído
    
                            (0.0,0 ), # Predio destruído incompleto    vertice 162
                            (0.4,0.0 ), # Predio destruído incompleto
                            (0.0, 0.3), # Predio destruído incompleto
    
                            (0.4, 0.0), # Predio destruído incompleto  vertice 165
                            (0, 0.3),  # Predio destruído incompleto    
                            (0.4, 0.3), # Predio destruído incompleto 
    
                            (0.0,0.3), # Predio destruído incompleto    vertice 168
                            (0.1, 0.3), # Predio destruído incompleto
                            (0.0, 0.4), # Predio destruído incompleto
    
                            (0.1,0.3), # Predio destruído incompleto    vertice 171
                            (0.2, 0.3), # Predio destruído incompleto
                            (0.2, 0.6), # Predio destruído incompleto
    
                            (0.4,0.3), # Predio destruído incompleto    vertice 174
                            (0.4, 0.5), # Predio destruído incompleto
                            (0.2, 0.3), # Predio destruído incompleto
    
                            (0, 0), # Sol Círculo           vertice 177
                            (0, 0), # Sol Círculo
                            (0, 0), # Sol Círculo
                            (0, 0), # Sol Círculo  
                            (0, 0), # Sol Círculo
                            (0, 0), # Sol Círculo
                            (0, 0), # Sol Círculo
                            (0, 0), # Sol Círculo  
    
                            (0, 0), # Sol Raios        vertice 185
                            (0, 0), # Sol Raios
                            (0, 0), # Sol Raios
                            (0, 0), # Sol Raios 
                            (0, 0), # Sol Raios
                            (0, 0), # Sol Raios
                            (0, 0), # Sol Raios
                            (0, 0), # Sol Raios
                            (0, 0), # Sol Raios
                            (0, 0), # Sol Raios
                            (0, 0), # Sol Raios 
                            (0, 0), # Sol Raios
                            (0, 0), # Sol Raios
                            (0, 0), # Sol Raios
                            (0, 0), # Sol Raios
                            (0, 0), # Sol Raios
    
                            (0, 0), # Nuvem          vertice 201
                            (0,0.185), # Nuvem
                            (-0.1,0.17), # Nuvem
                            (-0.13,0.13), # Nuvem 
                            (-0.2,0.10), # Nuvem
                            (-0.25,0.0), # Nuvem
                            (-0.2,-0.09), # Nuvem
                            (-0.1,-0.065), # Nuvem
                            (-0.05,-0.1), # Nuvem
                            (0.05,-.095), # Nuvem
                            (0.1,-0.08), # Nuvem
                            (0.16,-.091), # Nuvem
                            (0.23,-.04), # Nuvem
                            (0.25,.01), # Nuvem
                            (0.235,.06), # Nuvem
                            (0.255,.1), # Nuvem
                            (0.22,.165), # Nuvem
                            (0.15, .17), # Nuvem
                            (0.08, 0.15), # Nuvem
                            (0,0.185), # Nuvem
                        ]

########################## INSERE OS VÉRTICES DA CABEÇA DO MAGO ########################## 
pi = 3.14
counter = 0
radius = 0.065
angle = 0.0

for counter in range(n_verticesCabecaMago):
    angle += 2*pi/n_verticesCabecaMago 
    x = cos(angle)*radius
    y = sin(angle)*radius
    vertices[counter + num_v] = [x, y]
    
########################## INSERE OS VÉRTICES DO CORPO DO MAGO ########################## 
aCorpo = 0.1
bCorpo = 0.13
counter = 0
angle = 0.0
for counter in range((n_verticesCorpoMago)):
    angle += 2*pi/n_verticesCorpoMago
    x = cos(angle)*aCorpo
    y = sin(angle)*bCorpo - 0.2
    vertices[counter + num_v + n_verticesCabecaMago] = [x,y]
    
    
########################## INSERE OS VÉRTICES DA GEMA NO CETRO DO MAGO ########################## 
angle = 0.0
radius = 0.035
for counter in range(n_verticesCetroMago1):
    angle += 2*pi/n_verticesCetroMago1
    x = cos(angle)*radius + 0.15
    y = sin(angle)*radius + 0.1
    vertices[counter + 136] = [x, y]
    
########################## INSERE OS VÉRTICES DO SOL(metade de um círculo) ########################## 
angle = 2.95 
radius = 0.4
flag = 0
for counter in range(n_verticesSol0):
    angle += pi/n_verticesSol0
    x0 = cos(angle)*radius
    y0 = sin(angle)*radius
    vertices[counter+177] = [x0, y0]

    x1 = cos(angle) * 4.0
    y1 = sin(angle) * 4.0
    flag += 1
    if flag%3 == 0 and flag != 0:
        vertices[flag+185] = [0, 0]
        flag += 1
    vertices[flag-1+185] = [x1, y1]

### Requisita slot da para enviar dados da CPU para a GPU

In [19]:
# Request a buffer slot from GPU
buffer = glGenBuffers(1)
# Make this buffer the default one
glBindBuffer(GL_ARRAY_BUFFER, buffer)

### Conteúdo da variável vertice
parâmetros da função glBufferData: [https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glBufferData.xhtml]

In [20]:
# Upload data
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

### Associação de variáveis do programa GLSL (Vertex Shaders) com nossos dados
* 1: Definição do byte inicial e o offset dos dados

In [21]:
# Bind the position attribute
# --------------------------------------
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)

* 2: Solicita para a GPU a localização da variável 'position' (guarda a coordenada dos nossos vértices), que foi definida no Vertex Shader

In [22]:
loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)

* 3: A partir da localização anterior, indica-se à GPU onde está o conteúdo (via posições stride/offset) para a variável position (identificada na posição loc)

outros parâmetros:
* Define-se se possui duas coordenadas
* Que cada coordenada é do tipo float (GL_FLOAT)
* Que não se deve normalizar a coordenada (False)

mais detalhes: https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glVertexAttribPointer.xhtml

In [23]:
glVertexAttribPointer(loc, 2, GL_FLOAT, False, stride, offset)

### Atribui a localização da variável color (uniform)

In [24]:
loc_color = glGetUniformLocation (program, "color")

### Exibe-se a janela

In [25]:
glfw.show_window (window)

### Desenha Rua

In [26]:
def DesenhaRua():
    # Matriz básica de transformação, reposicionando em (0,0)
    mat_transform = np.array([    1.0,  0.0, 0, 0,
                                    0.0,  1.0, 0, 0,
                                    0.0,  0.0, 1.0 ,0,
                                     0, 0, 0, 1], np.float32)
    loc = glGetUniformLocation(program, "mat_transformation")
    
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transform)
    
    # Desenha a rua na parte de baixo da tela
    glUniform4f(loc_color, 0,0,0, 1.0) 
    glDrawArrays(GL_TRIANGLE_STRIP, 24, 4)

### Desenha mago

In [27]:
def DesenhaMago():
    global n_verticesCabecaMago, n_verticesCorpoMago, n_verticesCetroMago0
    global n_verticesCetroMago1
    global txMago, tyMago, cajadoRGB
    global sMago
    global rMago
    cos_d1 = cos(rMago)
    sin_d1 = sin(rMago)
    
    # Matriz de transformação do mago
    mat_rotation_x = np.array([     cos_d1, -sin_d1, 0.0, 0, 
                                    sin_d1,  cos_d1, 0.0, 0, 
                                    0.0,        0.0, 1.0, 0,
                                      0,          0,   0, 1], np.float32)
        
    mat_translation = np.array([    1.0,  0.0,  0, txMago, 
                                    0.0,  1.0,  0, tyMago, 
                                    0.0,  0.0, 1.0,     0,
                                      0,    0,   0,     1], np.float32)
    
    mat_scale = np.array([          sMago,  0.0, 0.0, 0,
                                    0.0   ,  sMago, 0.0, 0, 
                                    0.0   ,     0.0,   1, 0,
                                         0,       0,   0, 1], np.float32)
    
    
    
    mat_transform = multiplica_matriz(mat_rotation_x, mat_translation)
    mat_transform = multiplica_matriz(mat_scale, mat_transform)
    loc = glGetUniformLocation(program, "mat_transformation")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transform)
    
    # Desenha a cabeça do mago
    glUniform4f(loc_color, 1, 1, 1, 1.0) 
    glDrawArrays(GL_TRIANGLE_FAN, 36, n_verticesCabecaMago)
    
    # Desenha o corpo do mago
    glUniform4f(loc_color, 0.58, 0, 0.83, 1.0) 
    glDrawArrays(GL_TRIANGLE_FAN, 100, n_verticesCorpoMago)
    
    # Desenha o cajado do mago
    glUniform4f(loc_color, 0.294, 0.212, 0.13, 1.0) 
    glDrawArrays(GL_TRIANGLE_STRIP, 132, n_verticesCetroMago0)

    glUniform4f(loc_color, cajadoRGB[0], cajadoRGB[1], cajadoRGB[2], 1.0) 
    glDrawArrays(GL_TRIANGLE_FAN, 136, n_verticesCetroMago1)
    

### Desenha Quadrado Mágico

In [28]:
def DesenhaQuadradoMagico():
    global txMago, tyMago, cajadoRGB
    global quadradoMagico
    
    # Variáveis de teste para evitar problemas com float nas comparações 
    txMagoTeste = int(txMago * 100)
    tyMagoTeste = int(tyMago * 100)
    
    for i in range(len(quadradoMagico)):
        # Usa como matriz de translação o conteúdo da variável 'quadradoMagico' que contém as posições
        mat_translation = np.array([    1.0,  0.0,  0, quadradoMagico[i][0][0], 
                                        0.0,  1.0,  0, quadradoMagico[i][0][1], 
                                        0.0,  0.0, 1.0,     0,
                                          0,    0,   0,     1], np.float32)
        
        loc = glGetUniformLocation(program, "mat_transformation")
        glUniformMatrix4fv(loc, 1, GL_TRUE, mat_translation)
        
        #  Testa se o mago em um dos quadrados, comparando com a parte 
        # inteira da multiplicação por 100 de cada posição do quadrado.
        #  Se o mago se encontra acima do quadrado, a cor do quadrado E a cor da gema do
        # quadrado são brancas, muda a cor do quadrado para vermelho.
        if txMagoTeste == 15 and tyMagoTeste == -40:
            if testaParteInteira(cajadoRGB, [1,1,1]) and testaParteInteira(quadradoMagico[1][1], [1,1,1]):
                quadradoMagico[1][1] = [1,0,0]

        elif txMagoTeste == -9 and tyMagoTeste == -50:
            if testaParteInteira(cajadoRGB, [1,1,1]) and testaParteInteira(quadradoMagico[2][1], [1,1,1]):
                quadradoMagico[2][1] = [1,0,0]
        
        elif txMagoTeste == -60 and tyMagoTeste == -30:
            if testaParteInteira(cajadoRGB, [1,1,1]) and testaParteInteira(quadradoMagico[0][1], [1,1,1]):
                quadradoMagico[0][1] = [1,0,0]        
        
        # Seta Branco para se o mago não está acima do quadrado E o quadrado não é preto
        elif not testaParteInteira(quadradoMagico[i][1], [0,0,0]):
            quadradoMagico[i][1] = [1,1,1]            
        
        glUniform4f(loc_color, quadradoMagico[i][1][0], quadradoMagico[i][1][1], quadradoMagico[i][1][2], 1.0) 
        glDrawArrays(GL_TRIANGLE_STRIP, 15, 5)
    
    

### Desenha Sol

In [29]:
def DesenhaSol():
    global n_verticesTotal, n_verticesSol0, n_verticesSol1, n_verticesNuvem
    global txSol0, tySol0, txSol1, tySol1
    global sSol0, sSol1
    global rSol0, rSol1
    global quadradoMagico
    global raioSol
    
    # Matriz de transformação do Sol
    cos_d1 = cos(rSol0)
    sin_d1 = sin(rSol0)
        
    mat_rotation_x = np.array([     cos_d1, -sin_d1, 0.0, 0, 
                                    sin_d1,  cos_d1, 0.0, 0, 
                                    0.0,        0.0, 1.0, 0,
                                      0,          0,   0, 1], np.float32)
        
    mat_translation = np.array([    1.0,  0.0,  0, txSol0, 
                                    0.0,  1.0,  0, tySol0, 
                                    0.0,  0.0, 1.0,     0,
                                      0,    0,   0,     1], np.float32)
    
    mat_scale = np.array([          sSol0,  0.0, 0.0, 0,
                                    0.0   ,  sSol0, 0.0, 0, 
                                    0.0   ,     0.0,   1, 0,
                                         0,       0,   0, 1], np.float32)
    
    
    mat_transformSol0 = multiplica_matriz(mat_scale, mat_translationZero)
    mat_rot = multiplica_matriz(mat_rotation_x, mat_translationZero)
    mat_transformSol0 = multiplica_matriz(mat_rot, mat_transformSol0)
    mat_transformSol0 = multiplica_matriz(mat_translation, mat_transformSol0)
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformSol0) 
    
    # Desenha os raios de sol
    if not testaParteInteira(quadradoMagico[2][1], [0,0,0]):
        glUniform4f(loc_color, raioSol[1][0], raioSol[1][1], raioSol[1][2], 1.0) 
        glDrawArrays(GL_TRIANGLES, 185 , n_verticesSol1)
    
    # Desenha o Sol
    glUniform4f(loc_color, raioSol[0][0], raioSol[0][1], raioSol[0][2], 1.0) 
    glDrawArrays(GL_TRIANGLE_FAN, 177 , n_verticesSol0)

        

### Desenha janelas dos prédios
Função auxiliar que recebe o número de prédios, a matriz de transformação do prédio<br> e a cor RGB na forma [R,G,B], e a partir do vértice inferior esquerdo do prédio, desenha as janelas.

In [30]:
def DesenhaJanelas(numJanela, matTransform, RGB):

    mat_scale = np.array([          0.3,  0.0, 0.0, 0,
                                    0.0   ,  0.3, 0.0, 0, 
                                    0.0   ,     0.0,   1, 0,
                                         0,       0,   0, 1], np.float32) 
    ty = -0.7
    
    for i in range(1, numJanela + 1):
        tx = 0.725
        # A cada duas janelas desenhadas, reposiciona no eixo x para a esquerda
        if (i%2 == 0):
            tx -= 0.5
            
        mat_translation = np.array([    1.0,  0.0,   0, tx, 
                                        0.0,  1.0,   0, ty, 
                                        0.0,  0.0, 1.0,  0,
                                          0,    0,   0,  1], np.float32)
        # A cada duas janelas desenhadas, o terceiro reposiciona no eixo y para a cima
        if (i%2 == 0):
            ty += 0.5
        
        # A partir da matriz de transformação recebida na função, aplica as transformações
        mat_scaleWindow = multiplica_matriz(mat_scale, mat_translationZero)  
        mat_transform = multiplica_matriz(mat_scaleWindow, mat_translation)
        mat_transform = multiplica_matriz(matTransform, mat_transform)
        
        # São reutilizadas os vértices do quadrilátero usado para a base do prédio destruído.
        glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transform)        
        glUniform4f(loc_color, RGB[0], RGB[1], RGB[2], 1.0) 
        glDrawArrays(GL_TRIANGLES, 144, 6)
    

### Desenha prédio destruído

In [31]:
def DesenhaPredioDestruido():
    global n_verticesCabecaMago, num_v, n_verticesCorpoMago, n_verticesCetroMago0
    global n_verticesCetroMago1, n_verticesPredioDestruido0, n_verticesPredioDestruido1
    global txPredio0, tyPredio0, txPredio1, tyPredio1
    global sPredio0, sPredio1
    global rPredio0, rPredio1
    global quadradoMagico
    
    cos_d1 = cos(rPredio0)
    sin_d1 = sin(rPredio0)       
    
    mat_rotation_x = np.array([     cos_d1, -sin_d1, 0.0, 0, 
                                    sin_d1,  cos_d1, 0.0, 0, 
                                    0.0,        0.0, 1.0, 0,
                                      0,          0,   0, 1], np.float32)
        
    mat_translation = np.array([    1.0,  0.0,  0, txPredio0, 
                                    0.0,  1.0,  0, tyPredio0, 
                                    0.0,  0.0, 1.0,     0,
                                      0,    0,   0,     1], np.float32)
    
    mat_scale = np.array([          sPredio0,  0.0, 0.0, 0,
                                    0.0   ,  sPredio0, 0.0, 0, 
                                    0.0   ,     0.0,   1, 0,
                                         0,       0,   0, 1], np.float32)    
    
    mat_transformPredio0 = multiplica_matriz(mat_rotation_x, mat_translationZero)
    mat_rotationPredio0 = multiplica_matriz(mat_scale, mat_translationZero)  
    mat_transformPredio0 = multiplica_matriz(mat_rotationPredio0, mat_transformPredio0)
    mat_transformPredio0 = multiplica_matriz(mat_translation, mat_transformPredio0)
    
    cos_d1 = cos(rPredio1)
    sin_d1 = sin(rPredio1)
    
    mat_rotation_x = np.array([     cos_d1, -sin_d1, 0.0, 0, 
                                    sin_d1,  cos_d1, 0.0, 0, 
                                    0.0,        0.0, 1.0, 0,
                                      0,          0,   0, 1], np.float32)
        
    mat_translation = np.array([    1.0,  0.0,  0, txPredio1, 
                                    0.0,  1.0,  0, tyPredio1, 
                                    0.0,  0.0, 1.0,     0,
                                      0,    0,   0,     1], np.float32)
    
    mat_scale = np.array([          sPredio1,  0.0, 0.0, 0,
                                    0.0   ,  sPredio1, 0.0, 0, 
                                    0.0   ,     0.0,   1, 0,
                                         0,       0,   0, 1], np.float32)    
    
    mat_transformPredio1 = multiplica_matriz(mat_rotation_x, mat_translation)
    mat_transformPredio1 = multiplica_matriz(mat_scale, mat_transformPredio1)    
    
    loc = glGetUniformLocation(program, "mat_transformation")
    
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformPredio0)        
    glUniform4f(loc_color, 0.5, 0.3, 0.7, 1.0) 
    glDrawArrays(GL_TRIANGLES, 144, n_verticesPredioDestruido0)
    
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformPredio1)        
    glUniform4f(loc_color, 0.5, 0.3, 0.7, 1.0) 
    glDrawArrays(GL_TRIANGLES, 162, n_verticesPredioDestruido1)
    
    # Chama função que desenha janelas
    # Testa se todos os quadrados mágicos foram dissipados, antes de desenhar a criatura
    janelaRGB = [0,0,0]
    if testeAcabouQuadrado:
        janelaRGB = [1,1,0]
    DesenhaJanelas(4, mat_transformPredio0, janelaRGB)
    


### Desenha Nuvem
Desenha as nuvens na parte superior esquerda da tela

In [32]:
def DesenhaNuvem():
    global n_verticesTotal, n_verticesNuvem
    global txNuvem, tyNuvem
    global sNuvem
    global rNuvem
    global testeAcabouQuadrado
    
    loc = glGetUniformLocation(program, "mat_transformation")    
    
    # Desenha a nuvem mais escura e maior, no fundo
    cos_d1 = cos(rNuvem + pi)
    sin_d1 = sin(rNuvem + pi)    
    
    mat_rotation_x = np.array([     cos_d1, -sin_d1, 0.0, 0, 
                                    sin_d1,  cos_d1, 0.0, 0, 
                                    0.0,        0.0, 1.0, 0,
                                      0,          0,   0, 1], np.float32)
        
    mat_translation = np.array([    1.0,  0.0,  0, txNuvem-0.2, 
                                    0.0,  1.0,  0, tyNuvem+0.3, 
                                    0.0,  0.0, 1.0,     0,
                                      0,    0,   0,     1], np.float32)
    
    mat_scale = np.array([          1.5*sNuvem,  0.0, 0.0, 0,
                                    0.0   ,  1.5*sNuvem, 0.0, 0, 
                                    0.0   ,     0.0,   1, 0,
                                         0,       0,   0, 1], np.float32)    
    
    
    
    mat_transformNuvem = multiplica_matriz(mat_scale, mat_translationZero)
    mat_rotationNuvem = multiplica_matriz(mat_rotation_x, mat_translationZero)
    mat_transformNuvem = multiplica_matriz(mat_rotationNuvem, mat_transformNuvem)
    mat_transformNuvem = multiplica_matriz(mat_translation, mat_transformNuvem)
    
    # Testa se todos os quadrados mágicos foram dissipados, antes de desenhar a nuvem maligna
    if not testeAcabouQuadrado:
        glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformNuvem)   
        glUniform4f(loc_color, 0.3,0.3, 0.3, 1.0)
        glDrawArrays(GL_TRIANGLE_FAN, 201, n_verticesNuvem)
        
        
    # Desenha a nuvem meio escura e média, no meio das outras duas nuvens
    cos_d1 = cos(rNuvem+pi/15)
    sin_d1 = sin(rNuvem+pi/15)    
    
    mat_rotation_x = np.array([     cos_d1, -sin_d1, 0.0, 0, 
                                    sin_d1,  cos_d1, 0.0, 0, 
                                    0.0,        0.0, 1.0, 0,
                                      0,          0,   0, 1], np.float32)
        
    mat_translation = np.array([    1.0,  0.0,  0, txNuvem-0.4, 
                                    0.0,  1.0,  0, tyNuvem, 
                                    0.0,  0.0, 1.0,     0,
                                      0,    0,   0,     1], np.float32)
    
    mat_scale = np.array([          1.2*sNuvem,  0.0, 0.0, 0,
                                    0.0   ,  1.2*sNuvem, 0.0, 0, 
                                    0.0   ,     0.0,   1, 0,
                                         0,       0,   0, 1], np.float32)    
    
    
    
    mat_transformNuvem = multiplica_matriz(mat_scale, mat_translationZero)
    mat_rotationNuvem = multiplica_matriz(mat_rotation_x, mat_translationZero)
    mat_transformNuvem = multiplica_matriz(mat_rotationNuvem, mat_transformNuvem)
    mat_transformNuvem = multiplica_matriz(mat_translation, mat_transformNuvem)
    
    # Testa se todos os quadrados mágicos foram dissipados, antes de desenhar a criatura
    if not testeAcabouQuadrado:
        glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformNuvem)   
        glUniform4f(loc_color, 0.45, 0.45, 0.45, 1.0)
        glDrawArrays(GL_TRIANGLE_FAN, 201, n_verticesNuvem)    
    
    # Desenha a nuvem mais clara, na frente das outras duas núvens
    cos_d1 = cos(rNuvem)
    sin_d1 = sin(rNuvem)    
    
    mat_rotation_x = np.array([     cos_d1, -sin_d1, 0.0, 0, 
                                    sin_d1,  cos_d1, 0.0, 0, 
                                    0.0,        0.0, 1.0, 0,
                                      0,          0,   0, 1], np.float32)
        
    mat_translation = np.array([    1.0,  0.0,  0, txNuvem, 
                                    0.0,  1.0,  0, tyNuvem, 
                                    0.0,  0.0, 1.0,     0,
                                      0,    0,   0,     1], np.float32)
    
    mat_scale = np.array([          sNuvem,  0.0, 0.0, 0,
                                    0.0   ,  sNuvem, 0.0, 0, 
                                    0.0   ,     0.0,   1, 0,
                                         0,       0,   0, 1], np.float32)    
    
    mat_transformNuvem = multiplica_matriz(mat_scale, mat_translationZero)
    mat_rotationNuvem = multiplica_matriz(mat_rotation_x, mat_translationZero)
    mat_transformNuvem = multiplica_matriz(mat_rotationNuvem, mat_transformNuvem)
    mat_transformNuvem = multiplica_matriz(mat_translation, mat_transformNuvem)    
    
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformNuvem)    
    glUniform4f(loc_color, 0.6, 0.6, 0.6, 1.0)
    glDrawArrays(GL_TRIANGLE_FAN, 201, n_verticesNuvem)
    
    

### Desenha Roda
Desenha a roda gigante que o mago precisa diminuir e botar no canto.

Obs: O motivo de o objeto não possuir apenas uma matriz de transformação foi por conta da Roda
possuir três círculos, e por isso foram utilizados os vértices da cabeça do mago, com escalas diferentes. <br> Na hora de diminuir e aumentar a escala de cada círculo, um incremento de escala diferente é utilizado para cada círculo (de 10% da escala original). <br>
As matrizes de translação e rotação são as mesmas para as figuras.


In [33]:
def DesenhaRoda():
    global num_v, n_verticesCabecaMago
    global txRoda0, tyRoda0
    global sRoda0, sRoda1, sRoda2, sRoda3
    global rRoda0
    
    # Matriz de transformação da borracha da roda
    cos_d1 = cos(rRoda0)
    sin_d1 = sin(rRoda0)
    
    mat_rotation_x = np.array([     cos_d1, -sin_d1, 0.0, 0, 
                                    sin_d1,  cos_d1, 0.0, 0, 
                                    0.0,        0.0, 1.0, 0,
                                      0,          0,   0, 1], np.float32)
        
    mat_translation = np.array([    1.0,  0.0,   0, txRoda0, 
                                    0.0,  1.0,   0, tyRoda0, 
                                    0.0,  0.0, 1.0,       0,
                                      0,    0,   0,       1], np.float32)
    
    mat_scale = np.array([          sRoda0,     0.0, 0.0, 0,
                                    0.0   ,  sRoda0, 0.0, 0, 
                                    0.0   ,     0.0,   1, 0,
                                         0,       0,   0, 1], np.float32)
    
    mat_transformRoda0 = multiplica_matriz(mat_scale, mat_translationZero)
    mat_rotationRoda0 = multiplica_matriz(mat_rotation_x, mat_translationZero)  
    mat_transformRoda0 = multiplica_matriz(mat_rotationRoda0, mat_transformRoda0)
    mat_transformRoda0 = multiplica_matriz(mat_translation, mat_transformRoda0)
    
    # Matriz de transformação do aro da roda
    mat_scale = np.array([          sRoda1,     0.0, 0.0, 0,
                                    0.0   ,  sRoda1, 0.0, 0, 
                                    0.0   ,     0.0,   1, 0,
                                         0,       0,   0, 1], np.float32)
    
    
    mat_transformRoda1 = multiplica_matriz(mat_scale, mat_translationZero)
    mat_rotationRoda1 = multiplica_matriz(mat_rotation_x, mat_translationZero)    
    mat_transformRoda1 = multiplica_matriz(mat_rotationRoda0, mat_transformRoda1)
    mat_transformRoda1 = multiplica_matriz(mat_translation, mat_transformRoda1)
    
    # Matriz de transformação dos triângulos da roda
    mat_scale = np.array([          sRoda2,     0.0, 0.0, 0,
                                    0.0   ,  sRoda2, 0.0, 0, 
                                    0.0   ,     0.0,   1, 0,
                                         0,       0,   0, 1], np.float32)
    
    
    mat_transformRoda2 = multiplica_matriz(mat_scale, mat_translationZero)
    mat_rotationRoda2 = multiplica_matriz(mat_rotation_x, mat_translationZero)    
    mat_transformRoda2 = multiplica_matriz(mat_rotationRoda0, mat_transformRoda2)
    mat_transformRoda2 = multiplica_matriz(mat_translation, mat_transformRoda2)
    
    # Matriz de transformação do círculo no centro da roda
    mat_scale = np.array([          sRoda3,     0.0, 0.0, 0,
                                    0.0   ,  sRoda3, 0.0, 0, 
                                    0.0   ,     0.0,   1, 0,
                                         0,       0,   0, 1], np.float32)
    
    
    mat_transformRoda3 = multiplica_matriz(mat_scale, mat_translationZero)
    mat_rotationRoda3 = multiplica_matriz(mat_rotation_x, mat_translationZero)  
    mat_transformRoda3 = multiplica_matriz(mat_rotationRoda0, mat_transformRoda3)
    mat_transformRoda3 = multiplica_matriz(mat_translation, mat_transformRoda3)
    
    loc = glGetUniformLocation(program, "mat_transformation")    
    
    # Desenha a borracha da roda
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformRoda0)    
    glUniform4f(loc_color, 0.0, 0.1, 0.2, 1.0) 
    glDrawArrays(GL_TRIANGLE_FAN, num_v, n_verticesCabecaMago)
    
    # Desenha o aro da roda
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformRoda1) 
    glUniform4f(loc_color, 1,1,1, 1.0) 
    glDrawArrays(GL_TRIANGLE_FAN, num_v, n_verticesCabecaMago)
    
    # Desenha os triângulos escuros da roda
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformRoda2) 
    glUniform4f(loc_color, 0,0,0, 1.0) 
    glDrawArrays(GL_TRIANGLES, 0, 15)
    
    # Desenha o círculo escuro no centro da roda
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transformRoda3) 
    glUniform4f(loc_color, 0,0,0, 1.0) 
    glDrawArrays(GL_TRIANGLE_FAN, num_v, n_verticesCabecaMago)
    
    
    

### Desenha o cenário imóvel

In [34]:
def DesenhaCenarioImovel():
    global testeAcabouQuadrado
    mat_transform = np.array([    1.0,  0.0, 0, 0,
                                    0.0,  1.0, 0, 0,
                                    0.0,  0.0, 1.0 ,0,
                                     0, 0, 0, 1], np.float32)
    
    loc = glGetUniformLocation(program, "mat_transformation")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transform)    
    
    # Desenha o background de cor azul claro
    glUniform4f(loc_color, 0.1, 0.4, 1, 1.0) 
    glDrawArrays(GL_TRIANGLE_STRIP, 20, 4)
    
    
    #Desenha o prédio mais a direita no cenário
    glUniform4f(loc_color, 0.7, 0.5, 0.5, 1.0) 
    glDrawArrays(GL_TRIANGLE_STRIP, 28, 4)
    
    
    #Desenha as janelas do prédio mais a direita
    janelaRGB = [0,0,0]
    if testeAcabouQuadrado:
        janelaRGB = [1,1,0.2]
        
    mat_translationWindow = np.array([    1.0,  0.0,   0,  .7, 
                                          0.0,  1.0,   0, 0.05, 
                                          0.0,  0.0, 1.0,   0,
                                            0,    0,   0,   1], np.float32)
    
    DesenhaJanelas(10, mat_translationWindow, janelaRGB)    

### Loop principal da janela
Enquanto a janela não for fechada, o laço será executado. Onde se trabalha algumas interações com a OpenGL
A função glDrawArrays() recebe o tipo de primitiva (GL_POINTS), o índice inicial do array de vértices e a quantidade de vértices (len(vertices))

In [ ]:
glEnable(GL_COLOR_MATERIAL)

while not glfw.window_should_close(window):
    
    # funcao interna do glfw para gerenciar eventos de mouse, teclado, etc
    glfw.poll_events() 

    # limpa a cor de fundo da janela e preenche com outra no sistema RGBA
    glClear(GL_COLOR_BUFFER_BIT) 
    
    # definindo a cor da janela  
    glClearColor(0, 0, 0, 0)
    
    # seta a matriz de translação inicial
    mat_transform = np.array([  1.0,  0.0,   0, 0,
                                0.0,  1.0,   0, 0,
                                0.0,  0.0, 1.0, 0,
                                  0,    0,   0, 1], np.float32)
    loc = glGetUniformLocation(program, "mat_transformation")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transform)
    
    # Chama a função de desenha o background (céu + prédio à direita imóvel)
    DesenhaCenarioImovel()
    
    # Desenha o Sol
    DesenhaSol()
    
    # Desenha a Rua (imóvel)
    DesenhaRua()
            
    # Desenha a núvem
    DesenhaNuvem()
    
    # Desenha o prédio destruído
    DesenhaPredioDestruido()

    # Desenha os quadrados mágicos
    DesenhaQuadradoMagico()
    
    # Desenha a roda
    DesenhaRoda()
    
    # Desenha o mago na tela
    DesenhaMago()
    
    # gerencia troca de dados entre janela e o OpenGL
    glfw.swap_buffers(window)

glfw.terminate()